# Tutorial: Energy Densities of the Photon Targets

In this tutorial we will examine the integrated energy densities, $u [{\rm erg}\,{\rm cm}^{-3}]$, produced by the different photon fields that can be subject to inverse Compton scattering.

In [ ]:
import numpy as np
import astropy.units as u
from astropy.constants import M_sun
from astropy.coordinates import Distance
import matplotlib.pyplot as plt

In [ ]:
from agnpy.targets import SSDisk, SphericalShellBLR, RingDustTorus, PointSourceBehindJet
from agnpy.emission_regions import Blob
from agnpy.utils.plot import load_mpl_rc

# matplotlib adjustments
load_mpl_rc()
u_label = r"$u\,/\,{\rm erg}\,{\rm cm}^{-3}$"
u_prime_label = r"$u'\,/\,{\rm erg}\,{\rm cm}^{-3}$"
r_label = r"$r\,/\,{\rm cm}$"

## Shakura Sunyeav Disk

In [ ]:
M_BH = 1e9 * M_sun
L_disk = 1e46 * u.Unit("erg s-1")
eta = 1 / 12
R_tilde_in = 6
R_tilde_out = 200
disk = SSDisk(M_BH, L_disk, eta, R_tilde_in, R_tilde_out, R_g_units=True)
print(disk)

Let us define an array of distances over which we will calculate the integrated energy density $u$, using the `u` class function. Note that by default the energy density is computed in a stationary frame centred in the galaxy.

In [ ]:
r = np.logspace(15, 21) * u.cm
u_disk = disk.u(r)

In [ ]:
plt.loglog(r, u_disk)
plt.ylabel(u_label)
plt.xlabel(r_label)
plt.show()

Now let us simulate a point source behind the jet with the same luminosity as the disk. In the limit of large
distances the disk energy density should tend to the one of the point source. The point source behind the jet is monochromatic, we assume it has the same dimensionless energy as the photons emitted at the innermost disk radius.

In [ ]:
ps = PointSourceBehindJet(L_disk, disk.epsilon(disk.R_in))

In [ ]:
u_ps = ps.u(r)
plt.loglog(r, u_disk, label="SS Disk")
plt.loglog(r, u_ps, ls="--", label="point source behind jet")
plt.ylabel(u_label)
plt.xlabel(r_label)
plt.legend()
plt.show()

In [ ]:
print(u_ps / u_disk)

Let us see if the two energy densities also overlap in the case of large distances in the frame comoving with the blob. To compute the energy density in such a frame we just have to pass a `Blob` instance to the `u` function

In [ ]:
# set the spectrum normalisation (total energy in electrons in this case)
spectrum_norm = 1e48 * u.Unit("erg")
# define the spectral function through a dictionary
spectrum_dict = {
    "type": "PowerLaw",
    "parameters": {"p": 2.8, "gamma_min": 1e2, "gamma_max": 1e7},
}
R_b = 1e16 * u.cm
B = 1 * u.G
z = Distance(1e27, unit=u.cm).z
delta_D = 10
Gamma = 10
blob = Blob(R_b, z, delta_D, Gamma, B, spectrum_norm, spectrum_dict)

In [ ]:
u_prime_disk = disk.u(r, blob)
u_prime_ps = ps.u(r, blob)

In [ ]:
plt.loglog(r, u_prime_disk, label="SS Disk")
plt.loglog(r, u_prime_ps, ls="--", label="point source behind jet")
plt.ylabel(u_prime_label)
plt.xlabel(r_label)
plt.legend()
plt.show()

In [ ]:
print(u_prime_ps / u_prime_disk)

**Note** I think that the fact that the energy density ratio does not converge to 1 is due to the expression for the energy density of the disk not being properly normalised with respect to the radius variable $R$. One in fact can see how, given the same luminosity, changing the external and internal radii of the disk will alter the value of $u$ at large distances. This should not be the case as, no matter how the luminosity is geometrically distributed, it should always reduce to the case of a point source at very large distances.

## Spherical Shell Broad Line Region
Let us compute the energy density of the BLR in the two reference frame (stationary in the galaxy and comoving with the blob) and compare it with a monochromatic point source with the same properties of the BLR.

In [ ]:
blr = SphericalShellBLR(L_disk, 0.1, "Lyalpha", 1e17 * u.cm)
# point source with the same luminosity as the BLR
ps = PointSourceBehindJet(blr.xi_line * L_disk, blr.epsilon_line)

# compute the energy densities in the stationary frame
u_blr = blr.u(r)
u_ps = ps.u(r)

In [ ]:
plt.loglog(r, u_blr, label="spherical shell BLR")
plt.loglog(r, u_ps, ls="--", label="point source behind jet")
plt.legend()
plt.ylabel(u_label, fontsize=12)
plt.xlabel(r_label, fontsize=12)
plt.show()

In [ ]:
print(u_ps / u_blr)

In [ ]:
# compute the energy densities in teh frame comoving with the blob
u_prime_blr = blr.u(r, blob)
u_prime_ps = ps.u(r, blob)

In [ ]:
plt.loglog(r, u_prime_blr, label="spherical shell BLR")
plt.loglog(r, u_prime_ps, ls="--", label="point source behind jet")
plt.ylabel(u_prime_label, fontsize=12)
plt.xlabel(r_label, fontsize=12)
plt.legend()
plt.show()

In [ ]:
print(u_prime_ps / u_prime_blr)

As we can see in both reference frames the energy density of the BLR tends to the one of a point source behind the jet for large enough distances.

## Ring Dust Torus
Finally let us repeat the same exercise for the torus.

In [ ]:
dt = RingDustTorus(L_disk, 0.2, 1000 * u.K)
# point source approximating the torus
ps = PointSourceBehindJet(dt.xi_dt * L_disk, dt.epsilon_dt)

u_dt = dt.u(r)
u_ps = ps.u(r)

In [ ]:
plt.loglog(r, u_dt, label="ring dust torus")
plt.loglog(r, u_ps, ls="--", label="point source behind jet")
plt.ylabel(u_label)
plt.xlabel(r_label)
plt.legend()
plt.show()

In [ ]:
print(u_ps / u_dt)

In [ ]:
# compute the energy density in the comoving frame
u_prime_dt = dt.u(r, blob)
u_prime_ps = ps.u(r, blob)

In [ ]:
plt.loglog(r, u_prime_dt, label="ring dust torus")
plt.loglog(r, u_prime_ps, ls="--", label="point source behind jet")
plt.ylabel(u_prime_label)
plt.xlabel(r_label)
plt.legend()
plt.show()

In [ ]:
print(u_prime_ps / u_prime_dt)

Again in both reference frames the energy density of the dust torus tends to the one of a point source behind the jet with its same luminosity.